In [1]:
%matplotlib inline
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from metpy.units import units
from netCDF4 import num2date, Dataset
import numpy as np
from siphon.ncss import NCSS
import metpy.calc as mpcalc
from metpy.plots.ctables import colortables
from scipy.ndimage import gaussian_filter
from matplotlib import rcParams, colors

import warnings
warnings.filterwarnings('ignore')

In [41]:
def getdata(year=2001,month=4, day=12, hour=12, save=False):
    dt = datetime(year,month,day,hour)
    
    # Grab Pressure Level Data
    hght_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/hgt.{0:%Y%m}.nc'.format(dt))
    air_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                               'NARR/pressure/air.{0:%Y%m}.nc'.format(dt))
    #shum_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
    #                            'NARR/pressure/shum.{0:%Y%m}.nc'.format(dt))
    uwnd_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/uwnd.{0:%Y%m}.nc'.format(dt))
    vwnd_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/vwnd.{0:%Y%m}.nc'.format(dt))
    mslp_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                        'NARR/monolevel/prmsl.{0:%Y}.nc'.format(dt))
    uwnd10_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                          'NARR/monolevel/uwnd.10m.{0:%Y}.nc'.format(dt))
    vwnd10_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                          'NARR/monolevel/vwnd.10m.{0:%Y}.nc'.format(dt))
    
    
    #return {'Geopotential_Height':hght_data.variables['hgt'],'Air_Temperature':air_data.variables['air'],
    #        'Specific_Humidity':shum_data.variables['shum'],'U_wind':uwnd_data.variables['uwnd'],
    #        'V_wind':vwnd_data.variables['vwnd'],'lat':hght_data.variables['lat'],'lon':hght_data.variables['lon'],
    #        'time':hght_data.variables['time'],'level':hght_data.variables['level']}

#def plot(varname='Air_Temperature', level='850',):
    #data = x.widget.result
    #year = x.widget.kwargs['y']
    #month = x.widget.kwargs['m']

    # Pull out the lat and lon data
    lat = hght_data['lat'][:]
    lon = hght_data['lon'][:]
    
    plotcrs = ccrs.LambertConformal(central_latitude=45., central_longitude=-100.,
                                    standard_parallels=[30, 60])
    
    datacrs = ccrs.PlateCarree()

    tlatslons = plotcrs.transform_points(datacrs,lon,lat)
    tlon = tlatslons[:,:,0]
    tlat = tlatslons[:,:,1]
    
    #dt = datetime(year,month,day,hour)
    vtimes = num2date(hght_data['time'][:], units='hours since 1800-1-1 00:00:0.0')
    itime = np.where(vtimes==dt)[0][0]
    vtimes_mono = num2date(mslp_data['time'][:], units='hours since 1800-1-1 00:00:0.0')
    itime_mono = np.where(vtimes_mono==dt)[0][0]
    ilev_1000 = np.where(hght_data['level'][:]==1000)[0][0]
    ilev_850 = np.where(hght_data['level'][:]==850)[0][0]
    ilev_500 = np.where(hght_data['level'][:]==500)[0][0]
    ilev_300 = np.where(hght_data['level'][:]==300)[0][0]
    

    hght_1000 = gaussian_filter(hght_data.variables['hgt'][itime,ilev_1000,:,:],sigma=1.0) * units.meter
    hght_500 = gaussian_filter(hght_data.variables['hgt'][itime,ilev_500,:,:],sigma=1.0) * units.meter
    uwnd_500 = gaussian_filter(uwnd_data.variables['uwnd'][itime,ilev_500,:,:],sigma=1.0) * units('m/s')
    vwnd_500 = gaussian_filter(vwnd_data.variables['vwnd'][itime,ilev_500,:,:],sigma=1.0) * units('m/s')
    hght_850 = gaussian_filter(hght_data.variables['hgt'][itime,ilev_850,:,:],sigma=1.0) * units.meter
    uwnd_850 = gaussian_filter(uwnd_data.variables['uwnd'][itime,ilev_850,:,:],sigma=1.0) * units('m/s')
    vwnd_850 = gaussian_filter(vwnd_data.variables['vwnd'][itime,ilev_850,:,:],sigma=1.0) * units('m/s')
    tmpc_850 = (gaussian_filter(air_data.variables['air'][itime,ilev_850,:,:],sigma=1.0) * units('K')).to('degC')
    #shum_850 = shum_data.variables['shum'][itime,ilev_850,:,:] * units('kg/kg')
    hght_300 = gaussian_filter(hght_data.variables['hgt'][itime,ilev_300,:,:],sigma=1.0) * units.meter
    uwnd_300 = gaussian_filter(uwnd_data.variables['uwnd'][itime,ilev_300,:,:],sigma=1.0) * units('m/s')
    vwnd_300 = gaussian_filter(vwnd_data.variables['vwnd'][itime,ilev_300,:,:],sigma=1.0) * units('m/s')
    mslp = (gaussian_filter(mslp_data.variables['prmsl'][itime_mono,:,:],sigma=1.0) * units('Pa')).to('hPa')
    uwnd10 = gaussian_filter(uwnd10_data.variables['uwnd'][itime_mono,:,:],sigma=1.0) * units('m/s')
    vwnd10 = gaussian_filter(vwnd10_data.variables['vwnd'][itime_mono,:,:],sigma=1.0) * units('m/s')
    pcpin = gaussian_filter(hght_1000,sigma=1.0)*0.0
    
    dx, dy = mpcalc.lat_lon_grid_deltas(lon,lat)
    absvort_500 = mpcalc.absolute_vorticity(uwnd_500, vwnd_500, dx, dy, lat*units.degree, dim_order='yx')*1e5
    
    wspd_300 = mpcalc.get_wind_speed(uwnd_300,vwnd_300).to('kts')
    
    #e_850 = mpcalc.vapor_pressure(850.*units('hPa'),shum_850)
    #es_850 = mpcalc.saturation_vapor_pressure(tmpc_850.to('K'))
    #relh_850 = e_850/es_850*100
    
    # To set the map area, need to convert to proper coords.
    LL = plotcrs.transform_point(-125.,22.,ccrs.PlateCarree())
    UR = plotcrs.transform_point(-55.,52.,ccrs.PlateCarree())
    
     # Add state boundaries to plot
    states_provinces = cfeature.NaturalEarthFeature(category='cultural',
                                                    name='admin_1_states_provinces_lakes',
                                                    scale='50m', facecolor='none')
    # Add country borders to plot
    country_borders = cfeature.NaturalEarthFeature(category='cultural',
                                                   name='admin_0_countries',
                                                   scale='50m', facecolor='none')
    
    def plot_background(num,crs1):
        ax = plt.subplot(num,projection=crs1)
        #   ax.set_extent([west long, east long, south lat, north lat])
        ax.set_extent([LL[0],UR[0],LL[1],UR[1]],crs1)
        ax.coastlines('50m',edgecolor='black',linewidth=0.5)
        ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
        ax.add_feature(country_borders, edgecolor='black', linewidth=0.75)
        return ax
    
    clevpmsl = np.arange(800,1100,4)
    clev850 = np.arange(0,5000,30)
    clevrh700 = [50,70,80,90,100]
    clevtmpc850 = np.arange(-50,51,2)
    clev500 = np.arange(0,10000,60)
    clev300 = np.arange(0,15000,120)
    clevsped300 = np.arange(50,230,20)
    clevprecip = [0,0.01,0.03,0.05,0.10,0.15,0.20,0.25,0.30,0.40,0.50,
                  0.60,0.70,0.80,0.90,1.00,1.25,1.50,1.75,2.00,2.50]
    clevavor500 = list(range(-8, 1, 1))+list(range(8, 49, 2))
    colors1 = plt.cm.YlOrRd(np.linspace(0, 1, 31))
    colors2 = plt.cm.BuPu(np.linspace(0.5, 0.75, 8))
    colorsavor500 = np.vstack((colors2, (1, 1, 1, 1), colors1))
    
    fig=plt.figure(1,figsize=(22.,15.))
    fig.subplots_adjust(top=1, bottom=0, hspace=0.0, wspace=0.1)
    #fig.subplots_adjust(bottom=0, left=.01, right=.99, top=.99, hspace=.5, wspace = 0.2)
        
    # Following line is to get wind barbs properly on the correct projection
    # udat, vdat, xv, yv = m.transform_vector(uwnd_500[0,:,:],vwnd_500[0,:,:],tlon1,tlat1,15,21,returnxy=True)
    # Upper-left panel MSLP, 1000-500 hPa Thickness, Precip (in)
    ax1 = plot_background(221,plotcrs)
    ax1.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    plt.title('MSLP (hPa), 2m TMPF, and Precip',loc='left')
    plt.title('VALID: %s' %(dt),loc='right')
    cmap = colortables.get_colortable('precipitation')
    cf = ax1.contourf(tlon,tlat,pcpin,clevprecip,colors=cmap.colors)
    ax1.barbs(lon,lat,(uwnd10.to('kts')).m,(vwnd10.to('kts')).m,length=6,regrid_shape=15,
              transform=datacrs)
    cbar = plt.colorbar(cf,orientation='horizontal',extend='both',aspect=65,pad=0,extendrect='True')
    cs2 = ax1.contour(tlon,tlat,hght_500-hght_1000,clev500,colors='r',linewidths=1.5,linestyles='dashed')
    cs  = ax1.contour(tlon,tlat,mslp,clevpmsl,colors='k',linewidths=1.5)
    plt.clabel(cs,fontsize=10,inline=True,fmt='%d',rightside_up=True)
    plt.clabel(cs2,fontsize=9,inline=True,fmt='%d',rightside_up=True)
        
    # Upper-right panel 850-hPa Heights and Temp (C)
    ax2 = plot_background(222,plotcrs)
    ax2.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    cmap = plt.cm.coolwarm
    cf = ax2.contourf(tlon,tlat,tmpc_850,clevtmpc850,cmap=cmap,extend='both')
    cbar = plt.colorbar(cf,orientation='horizontal',extend='both',aspect=65,pad=0,extendrect='True')
    cs1 = ax2.contour(tlon,tlat,tmpc_850,clevtmpc850,colors='k',linewidths=1.5, linestyles=':',alpha=0.5)
    cs = ax2.contour(tlon,tlat,hght_850,clev850,colors='k',linewidths=1.5)
    ax2.barbs(lon,lat,(uwnd_850.to('kts')).m,(vwnd_850.to('kts')).m,length=6,regrid_shape=15,
              transform=datacrs)
    plt.clabel(cs,fontsize=10,inline=True,fmt='%d',rightside_up=True)
    plt.clabel(cs1,fontsize=10,inline=True,fmt='%d')
    plt.title('850-hPa HGHTs (m) and TMPC',loc='left')
    plt.title('VALID: %s' %(dt),loc='right')
        
    # Lower-left panel 500-hPa Heights and AVOR
    ax3 = plot_background(223,plotcrs)
    ax3.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    cf = ax3.contourf(tlon,tlat,absvort_500,clevavor500,colors=colorsavor500,extend='both')
    cs1 = ax3.contour(tlon,tlat,absvort_500,clevavor500,colors='k',linewidths=1.5,linestyles=':',alpha=0.5)
    cbar = plt.colorbar(cf,orientation='horizontal',extend='both',aspect=65,pad=0,extendrect='True')
    cs = ax3.contour(tlon,tlat,hght_500[:,:],clev500,colors='k',linewidths=1.5)
    ax3.barbs(lon,lat,(uwnd_500.to('kts')).m,(vwnd_500.to('kts')).m,length=6,regrid_shape=15,
              transform=datacrs)
    plt.clabel(cs,fontsize=10,inline=True,fmt='%d',rightside_up=True)
    plt.title('500-hPa HGHTs (m) and AVOR ($*10^5$ $s^{-1}$)',loc='left')
    plt.title('VALID: %s' %(dt),loc='right')
        
    # Lower-right panel 300-hPa Heights and Wind Speed (kts)
    ax4 = plot_background(224,plotcrs)
    ax4.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    cmap = plt.cm.get_cmap("BuPu")
    cf = ax4.contourf(tlon,tlat,wspd_300,clevsped300,cmap=cmap,extend='max')
    cbar = plt.colorbar(cf,orientation='horizontal',extend='max',aspect=65,pad=0,extendrect='True')
    cs = ax4.contour(tlon,tlat,hght_300,clev300,colors='k',linewidths=1.5)
    ax4.barbs(lon,lat,(uwnd_300.to('kts')).m,(vwnd_300.to('kts')).m,length=6,regrid_shape=15,
              transform=datacrs)
    plt.clabel(cs,fontsize=10,inline=True,fmt='%d',rightside_up=True)
    plt.title('300-hPa HGHTs (m) and SPED (kts)',loc='left')
    plt.title('VALID: %s' %(dt),loc='right')
    
    if save:
        plt.savefig('NARR_'+dt.strftime('%Y%m%d')+'_'+dt.strftime('%H00')+'.png',dpi=150)
        plt.close()
    else:
        plt.show()



In [3]:
year_widget = widgets.Dropdown(
    options=list(range(1979,2017)),
    description='Year', alignment='center')
month_widget = widgets.Dropdown(
    options=list(range(1,13)),
    description='Month', alignment='center')
day_widget = widgets.Dropdown(description='Day', options=list(range(1,32)))
hour_widget = widgets.Dropdown(description='Hour', options=[0,3,6,9,12,15,18,21])


In [42]:
x = widgets.interact_manual(getdata,year=year_widget,month=month_widget, day=day_widget, hour=hour_widget)

interactive(children=(Dropdown(description='Year', options=(1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 19…